<a href="https://colab.research.google.com/github/PhoenixCC0722/Journey_to_become_DataScientist/blob/main/3_1_2_data_cleaning_with_pandas_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning with Pandas

In this notebook we'll go through a few basic data cleaning steps that should be performed on all new datasets where necessary.

We'll go through the process with both the `orders` and `orderlines` datasets. You can then practice these skills by cleaning the `products` dataset yourself

In [1]:
import pandas as pd

In [2]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

Before we begin, let's create a copy of the `orders` and `orderlines` DataFrames. This way we are sure any of our changes won't affect the original DataFrames

In [3]:
orders_df = orders.copy()

orderlines_df = orderlines.copy()

One of the best ways to begin data cleaning is by exploring using `.info()`. This will tell us:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

By exploring the missing values and correcting any incorrect datatypes, we often come across inconsistencies in our data.

Beyond this, we should also have a **check for any duplicate rows**.

Let's first deal with the duplicates, as it's nice and easy, then we'll explore what `.info()` has to tell us.

## 1.&nbsp; Duplicates
We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method.

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [4]:
# orders_df
orders_df.duplicated().sum()

0

In [5]:
# orderlines_df
orderlines_df.duplicated().sum()

0

We have no duplicate rows in either DataFrame. Easy, there is no problem to solve. Normally though, if there were some duplicates, we'd drop the extra rows.

# 2.&nbsp; `.info()`

In [6]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [7]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [8]:
print(f"5 missing values represents {((orders_df.total_paid.isna().sum() / orders_df.shape[0])*100).round(5)}% of the rows in our DataFrame")

5 missing values represents 0.0022% of the rows in our DataFrame


A quick way to find out a percentage here, if you don't need to print out a sentence for yourself/students/colleagues is `.value_count(normalize=True)`

In [9]:
orders_df.total_paid.isna().value_counts(normalize=True)

False    0.999978
True     0.000022
Name: total_paid, dtype: float64

As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [10]:
orders_df = orders_df.loc[~orders.total_paid.isna(), :]

Should you have a significant number of missing values in the future, you have a choice:
+ you can impute the values
+ you can take the values from other DataFrames, if they are present there
+ you can delete the values
+ or any number of other creative solutions

Please, always consider how much time you have on your project, and what impact your method of choice will have on your final assesment.

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines_df`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [11]:
orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])

### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [12]:
orderlines_df["date"] = pd.to_datetime(orderlines_df["date"])

#### 4.1.2.&nbsp;`unit_price`

In [15]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

ValueError: Unable to parse string "1.137.99" at position 6

As you can see when we try to convert `unit_price` to a numerical datatype, we receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have 2 decimal points. Let's see if there are any other numbers like this.

In [16]:
orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()

False    257814
True      36169
Name: unit_price, dtype: int64

Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [17]:
two_dot_percentage = ((orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows as we only have 2 weeks for this project and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution?

Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [18]:
two_dot_order_ids_list = orderlines_df.loc[orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]
orderlines_df = orderlines_df.loc[~orderlines_df.id_order.isin(two_dot_order_ids_list)]

In [19]:
orderlines_df.shape[0]

216250

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [20]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

It worked perfectly

# Challenge: Clean the `products` DataFrame
Now it's your turn. Use the lessons you learnt above and clean the products DataFrame. You don't have to copy exactly what we did. Think about the consequences of your actions, sometimes it is ok to delete rows, other times you may wish to come up with more creative solutions.

In [21]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

In [22]:
# create a copy of the dataframe products,
#  this way we are sure any of our changes won't affect the orginal DataFrames
products_df = products.copy()
# to check the basic info of the dataframe
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


### Look for Duplicates

In [23]:
# check for the duplicates
products_df.duplicated().sum() #8746 duplicates

8746

In [24]:
# delete the dupllicated rows
products_df = products_df.drop_duplicates()

### Look for Missing values


In [25]:
# check for the general info of this dataframe
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10573 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10530 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


In [26]:
# desc, price, type have missing values
desc_missing = products_df.desc.isna().sum()
price_missing = products_df.price.isna().sum()
type_missing = products_df.type.isna().sum()
# the total rows of this frame
rows_sum =  products_df.shape[0]
print(f"{desc_missing} missing values in the column 'desc'", f"{price_missing} missing values in the column 'price'", f"{type_missing} missing values in the column 'type'", sep="\n")

7 missing values in the column 'desc'
46 missing values in the column 'price'
50 missing values in the column 'type'


In [28]:
# percentitle of th missing values in each column
pct_desc_missing = products_df.desc.isna().value_counts(normalize=True)
pct_price_missing = products_df.price.isna().value_counts(normalize=True)
pct_type_missing = products_df.type.isna().value_counts(normalize=True)
#print(f"{pct_desc_missing}", f"{pct_price_missing}", f"{pct_type_missing}", sep="\n")
print(f"{desc_missing} missing values represents {((desc_missing / rows_sum)*100).round(5)}% of the rows in the column 'desc'",f"{price_missing} missing values represents  {((price_missing / rows_sum)*100).round(5)}% of the rows in the column 'price'",f"{type_missing} missing values represents  {((type_missing / rows_sum)*100).round(5)}%  of the rows in the column 'type'",sep="\n")

7 missing values represents 0.06616% of the rows in the column 'desc'
46 missing values represents  0.43478% of the rows in the column 'price'
50 missing values represents  0.47259%  of the rows in the column 'type'


In [29]:
# the missing values are just tiny amount, we can delete them
products_df = products_df.loc[~products.desc.isna(), :]
products_df = products_df.loc[~products_df.price.isna(), :]
products_df = products_df.loc[~products_df.type.isna(), :]
products_df.info()
products_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10477 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10477 non-null  object
 1   name         10477 non-null  object
 2   desc         10477 non-null  object
 3   price        10477 non-null  object
 4   promo_price  10477 non-null  object
 5   in_stock     10477 non-null  int64 
 6   type         10477 non-null  object
dtypes: int64(1), object(6)
memory usage: 654.8+ KB


,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364


### Check / Change Data types

In [30]:
# check the info of the dataframe
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10477 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10477 non-null  object
 1   name         10477 non-null  object
 2   desc         10477 non-null  object
 3   price        10477 non-null  object
 4   promo_price  10477 non-null  object
 5   in_stock     10477 non-null  int64 
 6   type         10477 non-null  object
dtypes: int64(1), object(6)
memory usage: 654.8+ KB


In [31]:
# price, promo_price and the type should have the int64 Datatype
# check the 2 decimal points in the price column
products_df.price.str.contains("\d+\.\d+\.\d+").value_counts()

False    10104
True       373
Name: price, dtype: int64

In [32]:
# 427 rows have the 2 decimal points
two_dot_percentage = ((products_df.price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / products_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame") #price column

The 2 dot problem represents 3.56% of the rows in our DataFrame


In [33]:
# check the 2 decimal points in the promo_price column
products_df.promo_price.str.contains("\d+\.\d+\.\d+").value_counts()

False    5921
True     4556
Name: promo_price, dtype: int64

In [34]:
# 6362 rows have the 2 decimal points in promo_price
two_dot_percentage = ((products_df.promo_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / products_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame") #price column

The 2 dot problem represents 43.49% of the rows in our DataFrame


In [35]:
# price only have 2.2% 2 decimal points, therefore we can delete the rows
products_df = products_df.loc[~products_df.price.str.contains("\d+\.\d+\.\d+")]
products_df.shape[0]

10104

In [36]:
# price only have 2.2% 2 decimal points, therefore we can delete the rows
products_df = products_df.loc[~products.price.isna(), :]
products_df.shape[0]

10104

In [37]:
products_df["price"] = pd.to_numeric(products_df["price"])
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10104 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          10104 non-null  object 
 1   name         10104 non-null  object 
 2   desc         10104 non-null  object 
 3   price        10104 non-null  float64
 4   promo_price  10104 non-null  object 
 5   in_stock     10104 non-null  int64  
 6   type         10104 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 631.5+ KB


In [42]:
# 33.3% is way too much high, we have to do string parsing with regular expression
products_df = products.copy()
products_df.head()
products_df.iloc[48:57]

,sku,name,desc,price,promo_price,in_stock,type
48,IST0009,iStabilizer iPhone Tripod Mount,IPhone Tripod Adapter for photo and video.,19.99,199.892,0,1216
49,TWS0027,Twelve South BookBook Case MacBook Air / Pro 1...,Funda Macbook Air / Pro 13 inch vintage leathe...,79.99,749.898,0,13835403
50,APP0367,Apple Mini DisplayPort to DVI Adapter Mac dual...,Adapter Mini Display Port to DVI dual channel ...,119,1.119.976,0,1325
51,APP0344,"Apple Thunderbolt Display 27 ""Monitor Mac",Monitor Display 27-inch Apple Thunderbolt (MC9...,1149,10.449.923,0,1296
52,BLL0008,BlueLounge CableDrop Pack 6 Black,Cable management Mac iPhone iPad and iPod.,11.99,119.911,0,12285400
53,BLL0009,BlueLounge CableDrop Pack 6 White,Cable management Mac iPhone iPad and iPod.,12.99,109.868,0,12285400
54,BLL0011,BlueLounge Milo iPhone and iPod black support,Desktop support iPhone and iPod.,14.99,149.895,0,1216
55,BLL0012,Milo BlueLounge support iPhone and iPod white,Desktop support iPhone and iPod.,14.99,149.919,0,1216
56,KIN0002-2,Mac memory Kingston 2GB (2x1GB) 667MHz DDR2 SO...,RAM 2GB (2x1GB) Mac mini and iMac (2006/07) Ma...,37.98,289.904,0,1364


In [43]:
products_df.promo_price = products_df["promo_price"].str.replace(r'(\d+)\.(\d+)\.', r'\1\2.',regex=True)
products_df.price = products_df["price"].str.replace(r'(\d+)\.(\d+)\.', r'\1\2.',regex=True)
#products_df.head()
products_df.iloc[48:57]
#products_df.iloc[4:9]
# change the data type

,sku,name,desc,price,promo_price,in_stock,type
48,IST0009,iStabilizer iPhone Tripod Mount,IPhone Tripod Adapter for photo and video.,19.99,199.892,0,1216
49,TWS0027,Twelve South BookBook Case MacBook Air / Pro 1...,Funda Macbook Air / Pro 13 inch vintage leathe...,79.99,749.898,0,13835403
50,APP0367,Apple Mini DisplayPort to DVI Adapter Mac dual...,Adapter Mini Display Port to DVI dual channel ...,119,1119.976,0,1325
51,APP0344,"Apple Thunderbolt Display 27 ""Monitor Mac",Monitor Display 27-inch Apple Thunderbolt (MC9...,1149,10449.923,0,1296
52,BLL0008,BlueLounge CableDrop Pack 6 Black,Cable management Mac iPhone iPad and iPod.,11.99,119.911,0,12285400
53,BLL0009,BlueLounge CableDrop Pack 6 White,Cable management Mac iPhone iPad and iPod.,12.99,109.868,0,12285400
54,BLL0011,BlueLounge Milo iPhone and iPod black support,Desktop support iPhone and iPod.,14.99,149.895,0,1216
55,BLL0012,Milo BlueLounge support iPhone and iPod white,Desktop support iPhone and iPod.,14.99,149.919,0,1216
56,KIN0002-2,Mac memory Kingston 2GB (2x1GB) 667MHz DDR2 SO...,RAM 2GB (2x1GB) Mac mini and iMac (2006/07) Ma...,37.98,289.904,0,1364


In [ ]:
products_df["promo_price"] = pd.to_numeric(products_df["promo_price"])
products_df["price"] = pd.to_numeric(products_df["price"])
products_df.info()
# to check if this first decimal point were removed
products_df.loc[products_df['sku'].isin(['CRU0015-2','SYN0078','APP0672']),:]